In [1]:
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import math
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(asctime)s: %(message)s')

%matplotlib widget
# %matplotlib inline

import ipywidgets as widgets

%config InlineBackend.figure_format = 'retina'
# plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

### Single spike train analysis:
### ISI - Time difference between adjacent spikes
### TIH - Histogram of ISI
### Survivor function - The probability of surviving at least to time t (before an event) 
### $ Survivor(t) = 1 - CDF(ISI(t)) = 1 - \sum_{i=1}^{t}ISI(i) $
### Hazard function - The probability of an event at a time n after the previous event
### $ Hazard(t) = \frac{ISI(t)}{Survivor(t)} $
### .

### properties of the hazard function:
### $ h(t) \geq 0 $
### $ h(t) $ is not probability so $ \int_{}h(t) \longrightarrow \infty $
### very noisy for long ISIs

In [2]:
prob = 0.3
prob_vec = np.array([prob]*10)
ISI_vals = [prob]
rest = 1
for i in range(9):
    rest -= ISI_vals[i]
    ISI_vals.append(prob*rest)

CDF = np.hstack((0,np.cumsum(ISI_vals)[:-1]))
survivor = 1-CDF
hazard = ISI_vals/survivor

In [3]:
fig, ax = plt.subplots(nrows=5, figsize=(10,8))
ax[0].bar(np.arange(10), prob_vec)
ax[0].set_ylim([0, prob+0.1])
ax[0].set_title('probability of a spike')
ax[1].bar(np.arange(10), np.array(ISI_vals))
ax[1].set_ylim([0, prob+0.1])
ax[1].set_title('TIH')
ax[2].bar(np.arange(10), CDF)
ax[2].set_title('CDF(ISI)')
ax[3].bar(np.arange(10), survivor)
ax[3].set_title('Survivor function')
ax[4].bar(np.arange(10), hazard)
ax[4].set_ylim([0, prob+0.1])
ax[4].set_title('Hazard function')
fig.tight_layout()
for x in ax:
    x.set_xlabel('bins')
    x.set_ylabel('p')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# Generate a poisson neuron with refractory period
samp = 1000
rate = 20 / samp 
duration = 3600    
ref_period = 10 #in ms
spk_vec=np.zeros(duration*samp)
rand_vec=np.random.uniform(low=0, high=1, size=duration*samp)
idx=0
while idx<duration*samp:
    if rand_vec[idx] <= rate:
        spk_vec[idx] = 1
        idx = idx+ref_period+1
    else:
        idx=idx+1

# first we calculate TIH and TIH probabilities
spk_times = np.where(spk_vec==1)[0]
isi_vec = np.diff(spk_times)
tih, _ = np.histogram(isi_vec, bins=np.arange(1,400,1), density=True)

fig, ax = plt.subplots(figsize = (10,3))
ax.bar(np.arange(1,399,1),tih)
__ = ax.set_title('TIH')
ax.set_xlabel('Interval [ms]')
ax.set_ylabel('ISI(t)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
survivor = 1 - np.cumsum(tih)

fig, ax = plt.subplots(figsize = (10,3))
ax.bar(np.arange(1,399,1),survivor)
__ = ax.set_title('Survivor')
ax.set_xlabel('Interval [ms]')
ax.set_ylabel('Survivor(t-1)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# calculatin hazard function 
hazard = tih[1:] / survivor[0:-1]

#plotting hazard function
fig, ax = plt.subplots(figsize = (10,3))
ax.plot(hazard)
ax.set_title('Hazard function')
ax.set_xlabel('Interval [ms]')
ax.set_ylabel('ISI(t)/Survivor(t-1)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Autocorrelation definition:
### $ Q(x,y)[n] = \sum_{i=-\infty}^{\infty} x(i)y(i+n)   $

In [15]:
dt = 1/samp
vec_length = len(spk_vec)
autocorr = sig.correlate(spk_vec, spk_vec, mode='full', method='auto')
autocorr[(vec_length-1):(vec_length+1)] = 0

In [19]:
max_offset = 200
fig, ax = plt.subplots(figsize=(10,3))
ax.plot(np.arange( (-max_offset)/samp,(max_offset)/samp,dt),autocorr[vec_length-max_offset:vec_length+max_offset])
ax.set_title('autocorrelation poisson neuron with refractory period')
__ = ax.set_ylabel('count')
__ = ax.set_xlabel('offset(sec)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Autocorrelation is symmetric